<a href="https://colab.research.google.com/github/ElenaNtm/power-thefts-investigation/blob/main/hedno_SQL(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import pandas as pd 
import numpy as np
import math
import operator 
from datetime import datetime
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#from sklearn.metrics import confusion_matrix

In [4]:
#from sklearn.ensemble import HistGradientBoostingClassifier

In [5]:
#from sklearn.model_selection import train_test_split
#import sklearn

In [6]:
#from sklearn.preprocessing import OrdinalEncoder

!! Αν δεις και γεμιζει η RAΜ κανε επανεκκινηση χρονου εκτελεσης

# The Dataset

## Powerthefts - Target Variable

In [7]:
path = "/content/drive/MyDrive/HEDNO/PowerThefts_TRAIN.csv"
df_power_thefts = pd.read_csv(path, sep= ',')

In [8]:
df_power_thefts.drop(['Unnamed: 0.1','Unnamed: 0'], axis=1, inplace = True)

## Rest Train + Test

###Records

In [9]:
path = "/content/drive/MyDrive/HEDNO/Records_TRAIN.csv"
df_records_train = pd.read_csv(path, sep= '|')

In [10]:
path = "/content/drive/MyDrive/HEDNO/Records_TEST.csv"
df_records_test = pd.read_csv(path,sep='|')

###Representations

In [11]:
path = "/content/drive/MyDrive/HEDNO/Representations_TRAIN.csv"
df_representations_train = pd.read_csv(path, sep= '|')

In [12]:
path = "/content/drive/MyDrive/HEDNO/Representations_TEST.csv"
df_representations_test = pd.read_csv(path, sep= '|')

###Requests

In [13]:
path = "/content/drive/MyDrive/HEDNO/Requests_TRAIN.csv"
df_requests_train = pd.read_csv(path, sep= '|')

In [14]:
path = "/content/drive/MyDrive/HEDNO/Requests_TEST.csv"
df_requests_test = pd.read_csv(path, sep= '|')

###Consumptions

In [15]:
path = "/content/drive/MyDrive/HEDNO/Consumptions_TRAIN.csv"
df_consumptions_train = pd.read_csv(path, sep= '|')

In [16]:
path = "/content/drive/MyDrive/HEDNO/Consumptions_TEST.csv"
df_consumptions_test = pd.read_csv(path, sep= '|')

#SQL

##Representations:  group by ACCT_NBR & SUCCESSOR   order by HM/NIA ΕΝΑΡΞΗΣ  sum SUPPLIER and last SUPPLIER 

SQL: 
SELECT ACCT_NBR, SUCCESSOR, 
    SUM(SUPPLIER) as TOTAL_SUPPLIER, 
    LAST_VALUE(SUPPLIER) OVER (PARTITION BY ACCT_NBR, SUCCESSOR ORDER BY START_DATE) as LAST_SUPPLIER
FROM REPRESENTATIONS
GROUP BY ACCT_NBR, SUCCESSOR
ORDER BY START_DATE


In [17]:
# Calculate the last supplier by account and successor
last_supplier = df_representations_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['SUPPLIER'].last().reset_index()
last_supplier.columns = ['ACCT_NBR', 'SUCCESSOR', 'LAST_SUPPLIER']
#last_supplier.head()

In [18]:
#test
last_supplier_test = df_representations_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['SUPPLIER'].last().reset_index()
last_supplier_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'LAST_SUPPLIER']
#last_supplier_test.head()

In [19]:
# Calculate the total supplier 
total_supplier = df_representations_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['SUPPLIER'].nunique().reset_index()
total_supplier.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_SUPPLIER']
#total_supplier.head()

In [20]:
# test 
total_supplier_test = df_representations_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['SUPPLIER'].nunique().reset_index()
total_supplier_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_SUPPLIER']
#total_supplier_test.head()

In [21]:
#merge
result_rep = pd.merge(total_supplier, last_supplier, on=['ACCT_NBR', 'SUCCESSOR'])
result_rep = pd.merge(result_rep, df_representations_train, on=['ACCT_NBR', 'SUCCESSOR'])
#result_rep.head()

In [22]:
#test
result_rep_test = pd.merge(total_supplier_test, last_supplier_test, on=['ACCT_NBR', 'SUCCESSOR'])
result_rep_test = pd.merge(result_rep_test, df_representations_test, on=['ACCT_NBR', 'SUCCESSOR'])
#result_rep_test.head()

In [23]:
result_rep = result_rep.sort_values(by='START_DATE',  ascending= True)
#result_rep.head()

In [24]:
result_rep.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE
500832,000A31FE38C6F648845941B642E287FE020000006838AE...,1,1,Δ,Δ,NaN,1199-11-22,2001-07-17
547413,000A31FE38C6F648845941B642E287FE0200000071BF15...,1,1,Δ,Δ,NaN,1199-12-08,2001-04-19
742733,000A31FE38C6F648845941B642E287FE020000009A4781...,1,1,Δ,Δ,Κ,1983-12-31,2018-11-14
430457,000A31FE38C6F648845941B642E287FE020000005957B0...,1,1,Δ,Δ,Κ,1983-12-31,2018-11-01
781795,000A31FE38C6F648845941B642E287FE02000000A271B6...,1,1,Δ,Δ,NaN,1983-12-31,2022-09-16


In [25]:
#test
result_rep_test = result_rep_test.sort_values(by='START_DATE',  ascending= True)
result_rep_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE
57898,000A31FE38C6F648845941B642E287FE020000006D291A...,1,1,Δ,Δ,Δ,1983-12-31,2017-02-28
117124,000A31FE38C6F648845941B642E287FE02000000DB9213...,1,1,Δ,Δ,NaN,1984-01-01,2022-09-16
26637,000A31FE38C6F648845941B642E287FE0200000031D089...,1,1,Δ,Δ,Κ,1984-01-01,2019-11-15
6587,000A31FE38C6F648845941B642E287FE020000000C2164...,1,1,Δ,Δ,Δ,1984-01-01,2017-05-29
41236,000A31FE38C6F648845941B642E287FE020000004D2BCB...,1,1,Δ,Δ,NaN,1984-01-01,2022-09-16


get rid of the first 2 lines from train

In [26]:
result_rep = result_rep.iloc[2:]
#result_rep.head()

In [27]:
path = "/content/drive/MyDrive/HEDNO/Edo_ola/result_rep.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  result_rep.to_csv(f)

In [28]:
path = "/content/drive/MyDrive/HEDNO/Edo_ola/result_rep_test.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  result_rep_test.to_csv(f)

##Requests:  group by ACCT_NBR & SUCCESSOR order by REQUEST_DATE sum(REQUEST_TYPE)  and last REQUEST

SELECT ACCT_NBR, SUCCESSOR, 
    SUM(REQUEST_TYPE) as TOTAL_REQUEST_TYPE,
    LAST_VALUE(REQUEST_TYPE) OVER (PARTITION BY ACCT_NBR, SUCCESSOR ORDER BY REQUEST_DATE) as LAST_REQUEST_TYPE
FROM Requests
GROUP BY ACCT_NBR, SUCCESSOR
ORDER BY REQUEST_DATE

In [29]:
# Calculate the total request type by account and successor
#value_counts για total_requwst_type
total_request_type = df_requests_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['REQUEST_TYPE'].count().reset_index()
total_request_type.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_REQUEST_TYPE']
#total_request_type.head()

In [30]:
# test
total_request_type_test = df_requests_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['REQUEST_TYPE'].count().reset_index()
total_request_type_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_REQUEST_TYPE']
#total_request_type_test.head()

In [31]:
# Calculate the last request type by account and successor
last_request_type = df_requests_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['REQUEST_TYPE'].last().reset_index()
last_request_type.columns = ['ACCT_NBR', 'SUCCESSOR', 'LAST_REQUEST_TYPE']
#last_request_type.head()

In [32]:
# test
last_request_type_test = df_requests_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['REQUEST_TYPE'].last().reset_index()
last_request_type_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'LAST_REQUEST_TYPE']
#last_request_type_test.head()

In [33]:
#merge
result_req = pd.merge(total_request_type, last_request_type, on=['ACCT_NBR', 'SUCCESSOR'])
#result_req = pd.merge(result_req, df_requests_train, on=['ACCT_NBR', 'SUCCESSOR'])
#result_req.head()

In [34]:
#test
result_req_test = pd.merge(total_request_type_test, last_request_type_test, on=['ACCT_NBR', 'SUCCESSOR'])
#result_req = pd.merge(result_req, df_requests_train, on=['ACCT_NBR', 'SUCCESSOR'])
#result_req_test.head()

In [35]:
path = "/content/drive/MyDrive/HEDNO/Edo_ola/result_req.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  result_req.to_csv(f)

In [36]:
path = "/content/drive/MyDrive/HEDNO/Edo_ola/result_req_test.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  result_req_test.to_csv(f)

##Consumptions group by ACCT_NBR & SUCCESSOR   order by MEASUREMENT_DATE  we want the last 50  CSS_MS_HS_USE (consumptions) and SUM MS_METER_NBR  (Did not work finding the last 50 consumptions due to size)


SELECT ACCT_NBR, SUCCESSOR, SUM(MS_METER_NBR) as TOTAL_MS_METER_NBR
FROM Consumptions
GROUP BY ACCT_NBR, SUCCESSOR
ORDER BY MEASUREMENT_DATE CSS_MS_HS_USE
ORDER BY MEASUREMENT_DATE

In [37]:
#Sort by MEASUREMENT_DATE
#df_consumptions_train = df_consumptions_train.sort_values(by='MEASUREMENT_DATE', ascending=False)

In [38]:
#Group by ACCT_NBR, SUCCESSOR, and CSS_MS_HS_USE
#grouped = df_consumptions_train.groupby(['ACCT_NBR', 'SUCCESSOR', 'CSS_MS_HS_USE'])

In [39]:
#new df with last 50 rows for each group
#last_50 = grouped.apply(lambda x: x.head(50)).reset_index(drop=True)
#αργεί πάααρα πολύ να φορτώσει

In [40]:
#total MS_METER_NBR by ACCT_NBR and SUCCESSOR
total_ms_meter_nbr = df_consumptions_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['MS_METER_NBR'].nunique().reset_index()
total_ms_meter_nbr.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_MS_METER_NBR']
#total_ms_meter_nbr.head()

In [41]:
#test
total_ms_meter_nbr_test = df_consumptions_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['MS_METER_NBR'].nunique().reset_index()
total_ms_meter_nbr_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_MS_METER_NBR']
#total_ms_meter_nbr_test.head()

In [42]:
total_ms_meter_nbr=pd.merge(total_ms_meter_nbr,df_consumptions_train, on=['ACCT_NBR','SUCCESSOR'], how='left')
#total_ms_meter_nbr.head()

In [43]:
#test
total_ms_meter_nbr_test = pd.merge(total_ms_meter_nbr_test,df_consumptions_test, on=['ACCT_NBR','SUCCESSOR'], how='left')
#total_ms_meter_nbr_test.head()

In [44]:
total_ms_meter_nbr.drop(['MS_METER_NBR', 'CSS_MS_HS_USE'], axis=1)

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR,BS_RATE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,21,2018-11-30
1,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,21,2018-03-30
2,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,21,2019-12-02
3,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,21,2020-07-02
4,000A31FE38C6F648845941B642E287FE0200000000004A...,2,2,21,2020-09-17
...,...,...,...,...,...
9201389,000A31FE38C6F648845941B642E287FE02000000FFFFD8...,3,2,13,2021-10-04
9201390,000A31FE38C6F648845941B642E287FE02000000FFFFEB...,1,2,22,2022-05-23
9201391,000A31FE38C6F648845941B642E287FE02000000FFFFEB...,1,2,11,2022-06-30
9201392,000A31FE38C6F648845941B642E287FE02000000FFFFEB...,1,2,11,2022-05-23


In [45]:
total_ms_meter_nbr_test.drop(['MS_METER_NBR', 'CSS_MS_HS_USE'], axis=1)

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR,BS_RATE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,10,2019-10-02
1,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,10,2019-02-01
2,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,10,2020-10-05
3,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,10,2021-03-08
4,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,10,2021-02-05
...,...,...,...,...,...
718163,000A31FE38C6F648845941B642E287FE02000000FFFD63...,2,1,10,2021-12-20
718164,000A31FE38C6F648845941B642E287FE02000000FFFD63...,2,1,10,2021-10-11
718165,000A31FE38C6F648845941B642E287FE02000000FFFD63...,2,1,10,2018-04-17
718166,000A31FE38C6F648845941B642E287FE02000000FFFD63...,2,1,10,2020-10-12


In [46]:
# Sort by MEASUREMENT_DATE
result_con = total_ms_meter_nbr.sort_values(by='MEASUREMENT_DATE' , ascending= True)
#result_con.head()

In [47]:
# test
result_con_test = total_ms_meter_nbr_test.sort_values(by='MEASUREMENT_DATE' , ascending= True)
#result_con_test.head()

In [48]:
path = "/content/drive/MyDrive/HEDNO/Edo_ola/result_con.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  result_con.to_csv(f)

In [49]:
path = "/content/drive/MyDrive/HEDNO/Edo_ola/result_con_test.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  result_con_test.to_csv(f)